## Variable & Automatic Gradient Calculation

## Autograd
Autograd를 사용하면 backprop을 위한 미분 값을 자동으로 계산해준다

자동계산을 위해서 사용하는 변수는 torch.autograd에 있는 Variable

__data__ : Tensor형태의 데이터가 담심

__grad__ : Data가 거쳐온 layer에 대한 미분값이 축적

__grad_fn__ : 미분 값을 계산한 함수에 대한 정보(어떤 연산에 대한 backward를 진행했는지)

## 1. Import Required Libraries

In [2]:
import torch
from torch.autograd import Variable

## 2. Tensor vs Variable

### 1) Declaration

In [3]:
x_tensor = torch.Tensor(3,4)
x_tensor
x_tensor.data # 원래는 Variable로 감싸진 형태만 data를 받았는데 tensor도 이제 받는듯..?

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [4]:
x_variable = Variable(x_tensor)
x_variable
x_variable.data

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

### 2) Variable of a Variable

In [5]:
# .data -> wrapped tensor
x_variable.data

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [6]:
print(x_variable.grad) # 연산진행 X 따라서 None이 출력

None


In [7]:
# .requires grad -> whether variable requires gradient

print(x_variable.requires_grad)                        # 처음에 안넣어줘서 False뜬다 
x_variable = Variable(x_tensor, requires_grad = True)  # 그래디언트 계산이 돼고 안돼고를 설정
x_variable.requires_grad

False


True

## 3. Graph & Variables

In [8]:
# create graph

x = Variable(torch.FloatTensor(3,4), requires_grad=True)
y = x**2 + 4*x
z = 2*y + 3

x.requires_grad, y.requires_grad, z.requires_grad # gradient계산여부 다 True

(True, True, True)

In [9]:
# .backward(gradient, retain_graph, create_graph, retain_variables) 백프로파게이션 자동으로
# compute gradient of current variable w.r.t. graph leaves

loss = torch.rand(3,4)
z.backward(loss) # backward를 수행해야 gradient가 계산된다.

print(x.grad)
y.grad, z.grad # 다 x로 이루어진 함수라 leaf node인 x에 대한 grad만 저장한다

tensor([[7.8139, 5.0601, 2.6307, 6.7307],
        [0.4253, 5.2048, 2.6260, 4.7302],
        [2.0839, 5.1093, 5.5659, 4.1034]])


(None, None)

In [10]:
## 다른 예를 통해 확인
a = torch.ones(2,2)
print(a)

tensor([[1., 1.],
        [1., 1.]])


In [13]:
a = Variable(a, requires_grad = True)
print(a)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [14]:
## variable이 가진 세가지 (data, grad, fn)
print(a.data)
print(a.grad)        # 아무런 연산을 진행하지 않았기 때문에 gradient와 gradient function둘다 0
print(a.grad_fn)     

tensor([[1., 1.],
        [1., 1.]])
None
None


In [16]:
b = a+2
print(b)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [19]:
c = b**2
print(c)

tensor([[9., 9.],
        [9., 9.]], grad_fn=<PowBackward0>)


In [20]:
out = c.sum()
print(out)      # 이 아웃값을 토대로 backward를 진행해줘야 값이 출력됨

tensor(36., grad_fn=<SumBackward0>)


In [21]:
out.backward()

In [24]:
print(a.grad)    # 전에는 None이 떴는데 연산을 해줘서 이제 값이 뜸
print(a.grad_fn) # 여전히 None인 이유는 a가 직접적으로 수행한 연산이 없기 때문

tensor([[6., 6.],
        [6., 6.]])
None


In [26]:
print(b.grad)    # leaf node만 값이 나옴
print(b.grad_fn) # add연산에 대한 backward적용

None


In [27]:
print(c.grad)    # leaf node만 값이 나옴
print(c.grad_fn) # add연산에 대한 backward 적용

None


In [28]:
grad = torch.Tensor([0.1,1,10])
out.backward(grad)

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.